In [1]:
library(caret)
library(caretEnsemble)
library(tidyverse)
library(corrplot)

Loading required package: ggplot2

Loading required package: lattice


Attaching package: 'caretEnsemble'


The following object is masked from 'package:ggplot2':

    autoplot


-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v tibble  3.1.0     v dplyr   1.0.5
v tidyr   1.1.4     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1
v purrr   0.3.4     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x caretEnsemble::autoplot() masks ggplot2::autoplot()
x dplyr::filter()           masks stats::filter()
x dplyr::lag()              masks stats::lag()
x purrr::lift()             masks caret::lift()

corrplot 0.92 loaded



In [2]:
load("lending_2017.rda")
load("lending_2018.rda")

In [3]:
nrow(lending_2018)
length(lending_2018)

[1] 495275

[1] 152

In [4]:
str(lending_2018)

tibble [495,275 x 152] (S3: tbl_df/tbl/data.frame)
 $ id                                        : num [1:495275] NA NA 1.31e+08 1.31e+08 1.31e+08 ...
 $ member_id                                 : logi [1:495275] NA NA NA NA NA NA ...
 $ loan_amnt                                 : num [1:495275] NA NA 5000 15000 11200 25000 3000 17000 20000 19200 ...
 $ funded_amnt                               : num [1:495275] NA NA 5000 15000 11200 25000 3000 17000 20000 19200 ...
 $ funded_amnt_inv                           : num [1:495275] NA NA 5000 15000 11200 25000 3000 17000 20000 19200 ...
 $ term                                      : chr [1:495275] NA NA "36 months" "36 months" ...
 $ int_rate                                  : num [1:495275] NA NA 20.39 9.92 30.79 ...
 $ installment                               : num [1:495275] NA NA 187 483 368 ...
 $ grade                                     : chr [1:495275] NA NA "D" "B" ...
 $ sub_grade                                 : chr [1:495275] 

In [5]:
unique(lending_2018$loan_status)

[1] NA                   "Current"            "Fully Paid"        
[4] "Late (31-120 days)" "Late (16-30 days)"  "Charged Off"       
[7] "In Grace Period"    "Default"

In [6]:
summary(lending_2018)

       id            member_id        loan_amnt      funded_amnt   
 Min.   : 70102325   Mode:logical   Min.   : 1000   Min.   : 1000  
 1st Qu.:131267288   NA's:495275    1st Qu.: 8000   1st Qu.: 8000  
 Median :136195162                  Median :14000   Median :14000  
 Mean   :136087208                  Mean   :16025   Mean   :16025  
 3rd Qu.:141042201                  3rd Qu.:22000   3rd Qu.:22000  
 Max.   :145647287                  Max.   :40000   Max.   :40000  
 NA's   :33                         NA's   :33      NA's   :33     
 funded_amnt_inv     term              int_rate      installment     
 Min.   :  725   Length:495275      Min.   : 5.31   Min.   :  29.76  
 1st Qu.: 8000   Class :character   1st Qu.: 8.46   1st Qu.: 254.56  
 Median :14000   Mode  :character   Median :11.80   Median : 386.82  
 Mean   :16022                      Mean   :12.73   Mean   : 466.61  
 3rd Qu.:22000                      3rd Qu.:16.01   3rd Qu.: 629.04  
 Max.   :40000                      

### Nos quitamos las filas y columnas que tengan +50% de NA's

In [7]:
lending_2018 <- lending_2018[which(rowMeans(!is.na(lending_2018)) > 0.5), which(colMeans(!is.na(lending_2018)) > 0.5)]

In [8]:
nrow(lending_2018)
length(lending_2018)

[1] 495242

[1] 109

In [9]:
summary(lending_2018)

       id              loan_amnt      funded_amnt    funded_amnt_inv
 Min.   : 70102325   Min.   : 1000   Min.   : 1000   Min.   :  725  
 1st Qu.:131267288   1st Qu.: 8000   1st Qu.: 8000   1st Qu.: 8000  
 Median :136195162   Median :14000   Median :14000   Median :14000  
 Mean   :136087208   Mean   :16025   Mean   :16025   Mean   :16022  
 3rd Qu.:141042201   3rd Qu.:22000   3rd Qu.:22000   3rd Qu.:22000  
 Max.   :145647287   Max.   :40000   Max.   :40000   Max.   :40000  
                                                                    
     term              int_rate      installment         grade          
 Length:495242      Min.   : 5.31   Min.   :  29.76   Length:495242     
 Class :character   1st Qu.: 8.46   1st Qu.: 254.56   Class :character  
 Mode  :character   Median :11.80   Median : 386.82   Mode  :character  
                    Mean   :12.73   Mean   : 466.61                     
                    3rd Qu.:16.01   3rd Qu.: 629.04                     
          

In [10]:
lending_2018$loan_status <- factor(lending_2018$loan_status)

In [11]:
str(lending_2018)

tibble [495,242 x 109] (S3: tbl_df/tbl/data.frame)
 $ id                        : num [1:495242] 1.31e+08 1.31e+08 1.31e+08 1.31e+08 1.31e+08 ...
 $ loan_amnt                 : num [1:495242] 5000 15000 11200 25000 3000 17000 20000 19200 6500 10000 ...
 $ funded_amnt               : num [1:495242] 5000 15000 11200 25000 3000 17000 20000 19200 6500 10000 ...
 $ funded_amnt_inv           : num [1:495242] 5000 15000 11200 25000 3000 17000 20000 19200 6500 10000 ...
 $ term                      : chr [1:495242] "36 months" "36 months" "60 months" "60 months" ...
 $ int_rate                  : num [1:495242] 20.39 9.92 30.79 21.85 7.34 ...
 $ installment               : num [1:495242] 186.8 483.4 367.8 688.4 93.1 ...
 $ grade                     : chr [1:495242] "D" "B" "G" "D" ...
 $ sub_grade                 : chr [1:495242] "D4" "B2" "G1" "D5" ...
 $ emp_title                 : chr [1:495242] "General Manager" "IT Director" "Client services" "Asphalt Supervisor" ...
 $ emp_length        

In [12]:
summary(lending_2018)

       id              loan_amnt      funded_amnt    funded_amnt_inv
 Min.   : 70102325   Min.   : 1000   Min.   : 1000   Min.   :  725  
 1st Qu.:131267288   1st Qu.: 8000   1st Qu.: 8000   1st Qu.: 8000  
 Median :136195162   Median :14000   Median :14000   Median :14000  
 Mean   :136087208   Mean   :16025   Mean   :16025   Mean   :16022  
 3rd Qu.:141042201   3rd Qu.:22000   3rd Qu.:22000   3rd Qu.:22000  
 Max.   :145647287   Max.   :40000   Max.   :40000   Max.   :40000  
                                                                    
     term              int_rate      installment         grade          
 Length:495242      Min.   : 5.31   Min.   :  29.76   Length:495242     
 Class :character   1st Qu.: 8.46   1st Qu.: 254.56   Class :character  
 Mode  :character   Median :11.80   Median : 386.82   Mode  :character  
                    Mean   :12.73   Mean   : 466.61                     
                    3rd Qu.:16.01   3rd Qu.: 629.04                     
          

In [13]:
unique(lending_2018["debt_settlement_flag"])

debt_settlement_flag
<chr>
N
Y


In [14]:
to_date <- c("issue_date")
to_factor <- c(
               "term", "grade", "home_ownership",
               "verification_status", "pymnt_plan",
               "inq_last_6mths",
               "initial_list_status", "collections_12_mths_ex_med",
               "application_type", "acc_now_delinq",
               "chargeoff_within_12_mths",
               "num_tl_30dpd", "pub_rec_bankruptcies", "hardship_flag",
               "disbursement_method", "debt_settlement_flag"
               )
to_ignore <- c("id", "url", "title", "zipcode",
              "earliest_cr_line", "last_pymnt_d",
              "next_pymnt_d", "last_credit_pull_d", "policy_code",
              "num_tl_120dpd_2m", "sub_grade", "emp_title", "emp_length",
              "purpose", "addr_state")

In [15]:
lending_2018 <- lending_2018[, !(names(lending_2018) %in% to_ignore)]
lending_2018[, to_factor] <- lapply(lending_2018[, to_factor], factor)
lending_2018[, to_date] <- lapply(lending_2018[, to_date], as.Date)

In [16]:
summary(lending_2018)

   loan_amnt      funded_amnt    funded_amnt_inv        term       
 Min.   : 1000   Min.   : 1000   Min.   :  725   36 months:344671  
 1st Qu.: 8000   1st Qu.: 8000   1st Qu.: 8000   60 months:150571  
 Median :14000   Median :14000   Median :14000                     
 Mean   :16025   Mean   :16025   Mean   :16022                     
 3rd Qu.:22000   3rd Qu.:22000   3rd Qu.:22000                     
 Max.   :40000   Max.   :40000   Max.   :40000                     
                                                                   
    int_rate      installment      grade       home_ownership  
 Min.   : 5.31   Min.   :  29.76   A:135177   ANY     :   488  
 1st Qu.: 8.46   1st Qu.: 254.56   B:141365   MORTGAGE:239220  
 Median :11.80   Median : 386.82   C:126850   OWN     : 60128  
 Mean   :12.73   Mean   : 466.61   D: 69046   RENT    :195406  
 3rd Qu.:16.01   3rd Qu.: 629.04   E: 18958                    
 Max.   :30.99   Max.   :1670.15   F:  3175                    
        

In [17]:
rf <- train(loan_status ~ ., data = lending_2018, method = "rf")
rf